<a href="https://colab.research.google.com/github/Justabhi96/NLP/blob/master/05_Text_feature_extraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import pandas as pd

In [0]:
df = pd.read_csv("smsspamcollection.tsv", sep = "\t")
df.head()

,label,message,length,punct
0,ham,"Go until jurong point, crazy.. Available only ...",111,9
1,ham,Ok lar... Joking wif u oni...,29,6
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,155,6
3,ham,U dun say so early hor... U c already then say...,49,6
4,ham,"Nah I don't think he goes to usf, he lives aro...",61,2


In [0]:
df.isnull().sum()

label      0
message    0
length     0
punct      0
dtype: int64

In [0]:
df["label"].value_counts()

ham     4825
spam     747
Name: label, dtype: int64

In [0]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer

In [0]:
X = df["message"]
y = df["label"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.33, random_state = 42)

In [0]:
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(X_train)
X_test_counts = count_vect.transform(X_test)

In [0]:
X_train_counts, X_train_counts.shape

(<3733x7082 sparse matrix of type '<class 'numpy.int64'>'
 	with 49992 stored elements in Compressed Sparse Row format>, (3733, 7082))

In [0]:
X_train.shape

(3733,)

In [0]:
tfidf_trans = TfidfTransformer()
X_train_tfidf = tfidf_trans.fit_transform(X_train_counts)
X_test_tfidf = tfidf_trans.transform(X_test_counts)

In [0]:
X_train_tfidf.shape

(3733, 7082)

####Since it is so common to perform count vectorizer followed by TfIdf transformer. SkLearn has given a way to perform both in one go using 

###TfidfVectorizer

In [0]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [0]:
tfidf_vect = TfidfVectorizer()
X_train_tfidf = tfidf_vect.fit_transform(X_train)
X_test_tfidf = tfidf_vect.transform(X_test)

In [0]:
X_train_tfidf.shape

(3733, 7082)

In [0]:
from sklearn.svm import LinearSVC

In [0]:
svc = LinearSVC()
svc.fit(X_train_tfidf, y_train)

LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
          intercept_scaling=1, loss='squared_hinge', max_iter=1000,
          multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
          verbose=0)

###Create a pipeline to perform all the things

In [0]:
from sklearn.pipeline import Pipeline

In [0]:
clf_pipeline = Pipeline([('tfidf', TfidfVectorizer()), ('svc', LinearSVC())])
clf_pipeline.fit(X_train, y_train)

Pipeline(memory=None,
         steps=[('tfidf',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=True, max_df=1.0, max_features=None,
                                 min_df=1, ngram_range=(1, 1), norm='l2',
                                 preprocessor=None, smooth_idf=True,
                                 stop_words=None, strip_accents=None,
                                 sublinear_tf=False,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, use_idf=True,
                                 vocabulary=None)),
                ('svc',
                 LinearSVC(C=1.0, class_weight=None, dual=True,
                           fit_intercept=True, intercept_scaling=1,
               

In [0]:
pred = clf_pipeline.predict(X_test)

In [0]:
from sklearn.metrics import confusion_matrix, classification_report

In [0]:
print(confusion_matrix(y_test, pred))

[[1586    7]
 [  12  234]]


In [0]:
print(classification_report(y_test, pred))

              precision    recall  f1-score   support

         ham       0.99      1.00      0.99      1593
        spam       0.97      0.95      0.96       246

    accuracy                           0.99      1839
   macro avg       0.98      0.97      0.98      1839
weighted avg       0.99      0.99      0.99      1839



In [0]:
clf_pipeline.predict(["Hi! how are you doing today"])

array(['ham'], dtype=object)

In [0]:
clf_pipeline.predict(["Congrats you have been selected as a winner. text back to win free tickets"])

array(['spam'], dtype=object)